### "Langchain 강의를 듣고 아쉬웠던 2%, 나에게 필요한 내용만 추가한 요약집"

### (1) API 키 로딩 (dotenv)
필수! API 키 같은 민감 정보는 `.env` 파일로 관리하고, `load_dotenv()`로 불러옴. 깔끔함.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from glob import glob
from pprint import pprint # 결과 보기 좋게 출력

### (2) 벡터 저장소 로드
미리 만들어둔 ChromaDB 벡터 저장소에서 임베딩된 문서들 불러옴. RAG의 핵심 재료임.
- 임베딩 모델: `text-embedding-3-small` (가성비 좋음)
- 컬렉션 이름: `chroma_test`
- 저장 경로: `./chroma_db`

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory="./chroma_db", # 미리 저장해둔 DB 경로
    )

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

벡터 저장소에 저장된 문서 수: 5


## 2. LCEL의 힘: 손쉽게 체인 만들기
LangChain Expression Language (LCEL)은 파이프(`|`) 연산자를 사용해 다양한 컴포넌트(프롬프트, 모델, 파서 등)를 유연하게 연결할 수 있게 해줌.

### 2.1 프롬프트 + LLM: 기본 중의 기본
가장 기본적인 체인 구성. 프롬프트 템플릿을 만들고 LLM과 연결함.

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 모델 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0.3, # 답변의 창의성 조절 (낮을수록 결정적)
    max_tokens=100, # 최대 답변 길이
    )

# 프롬프트 메시지 리스트 정의 (시스템 메시지, 사용자 메시지)
messages = [
    ("system", "You are a helpful assistant."), # 시스템 역할 부여
    ("user", "{query}"), # 사용자 질문 템플릿
]

# 메시지 리스트로부터 ChatPromptTemplate 생성
prompt = ChatPromptTemplate.from_messages(messages)

# 생성된 프롬프트 템플릿 구조 확인
print(prompt)

input_variables=['query'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]


In [5]:
# 프롬프트 템플릿이 어떤 입력 변수를 사용하는지 확인
print(prompt.input_variables)

['query']


In [6]:
# 템플릿에 실제 값(`query`)을 넣어 프롬프트 텍스트 완성 (렌더링)
prompt_text = prompt.format(query="테슬라 창업자는 누구인가요?")
print(prompt_text)

System: You are a helpful assistant.
Human: 테슬라 창업자는 누구인가요?


In [7]:
# 완성된 프롬프트 텍스트를 LLM에 직접 입력하여 응답 받기 (LCEL 체인 사용 전)
response_from_llm_direct = llm.invoke(prompt_text)

# LLM 응답(AIMessage 객체)에서 내용(content)만 추출하여 출력
print(response_from_llm_direct.content)

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하르드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프레처(JB Straubel), 이안 라이트(Ian Wright) 등입니다. 그러나 엘론 머스크가 테슬라의 CEO로서 가장 잘 알려져 있으며, 회사의 비전과 방향성을 주도해왔습니다. 테슬라는


In [8]:
# LCEL을 사용한 체인 구성: 프롬프트와 LLM을 `|` (파이프) 연산자로 연결. 이게 핵심!
chain = prompt | llm

# 구성된 체인 정보 출력 (어떤 컴포넌트들이 연결되었는지 보여줌)
print(chain)

first=ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]) last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001CB50830A50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001CB50792B90>, model_name='gpt-4o-mini', temperature=0.3, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=100)


In [9]:
# 체인의 입력 스키마 확인 (어떤 입력을 받는지 JSON 스키마 형태로 보여줌)
pprint(chain.input_schema.schema())

{'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'PromptInput',
 'type': 'object'}


In [10]:
# 체인 실행 방법 1: 딕셔너리 형태로 입력 (입력 변수 이름을 키로 사용)
response_from_chain_dict = chain.invoke({"query":"테슬라 창업자는 누구인가요?"})

# 체인 응답(AIMessage 객체)의 내용 출력
print(response_from_chain_dict.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)가 아닙니다. 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 그러나 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 성장에 중요한 역할을 하게 되었습니다. 이후 그는 테슬라의 얼굴


In [11]:
# 체인 실행 방법 2: 입력 변수가 하나일 경우, 문자열로 직접 입력 가능 (간편함)
response_from_chain_str = chain.invoke("테슬라 창업자는 누구인가요?") # 이 response_from_chain_str은 아래 Output Parser에서 사용됨

# 체인 응답(AIMessage 객체)의 내용 출력
print(response_from_chain_str.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 처음 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 성장에 큰 영향을 미쳤습니다. 현재 엘론 머스크는 테슬라


### 2.2 출력 파서: 원하는 형태로 결과 받기
LLM의 응답(주로 AIMessage 객체)을 우리가 원하는 포맷(문자열, JSON 등)으로 변환해주는 역할. 체인의 마지막에 연결함.

#### a) 문자열 파싱 (StrOutputParser)
LLM 응답(AIMessage 객체)에서 실제 텍스트 내용만 깔끔하게 뽑아줌. 제일 흔하게 씀.

In [12]:
# 이전 셀에서 실행한 체인의 결과 (AIMessage 객체)
response_from_chain_str # 이 변수는 위에서 `chain.invoke("테슬라 창업자는 누구인가요?")`로 얻은 AIMessage 객체임.

AIMessage(content='테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 처음 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 성장에 큰 영향을 미쳤습니다. 현재 엘론 머스크는 테슬라', response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 27, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_54eb4bd693', 'finish_reason': 'length', 'logprobs': None}, id='run-ea3aa5a3-da1f-4c01-bfb6-180094cf96ee-0', usage_metadata={'input_tokens': 27, 'output_tokens': 100, 'total_tokens': 127})

In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

# AIMessage 객체를 StrOutputParser에 통과시키면 문자열 내용만 반환됨
output_parser.invoke(response_from_chain_str)

'테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 처음 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 성장에 큰 영향을 미쳤습니다. 현재 엘론 머스크는 테슬라'

In [14]:
# 체인에 StrOutputParser 연결: prompt | llm | output_parser
str_chain = prompt | llm  | output_parser

query = "리비안의 설립년도는 언제인가요?"
str_response = str_chain.invoke(query)

print(str_response)
print(type(str_response)) # 타입이 문자열(str)인지 확인

리비안(Rivian)은 2009년에 설립되었습니다. 이 회사는 전기차 제조업체로, 주로 전기 픽업트럭과 SUV를 개발하고 있습니다.
<class 'str'>


#### b) JSON 출력 (JsonOutputParser)
LLM이 JSON 형식 문자열을 주면, 이걸 파이썬 딕셔너리로 변환해줌. LLM에게 JSON으로 달라고 요청해야 함.

In [15]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

# 기본 체인 (prompt | llm)을 사용. LLM에게 JSON 형식으로 출력하라고 요청함.
json_response_from_llm = chain.invoke("테슬라 창업자는 누구인가요? JSON 형식으로 출력해주세요.") 
print(json_response_from_llm) # AIMessage 객체, content 안에 JSON 문자열이 들어있음

# JsonOutputParser로 AIMessage의 content (JSON 문자열)를 파싱하여 파이썬 딕셔너리로 변환
json_parser_output = json_parser.invoke(json_response_from_llm)
print(json_parser_output)
print(type(json_parser_output)) # 타입이 딕셔너리(dict)인지 확인

content='```json\n{\n  "창업자": "엘론 머스크",\n  "설명": "테슬라의 공동 창립자이자 CEO로, 2004년에 회사에 합류했습니다."\n}\n```' response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 34, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_54eb4bd693', 'finish_reason': 'stop', 'logprobs': None} id='run-efb48a74-8a4d-4448-a21a-abd68b1aa291-0' usage_metadata={'input_tokens': 34, 'output_tokens': 47, 'total_tokens': 81}
{'창업자': '엘론 머스크', '설명': '테슬라의 공동 창립자이자 CEO로, 2004년에 회사에 합류했습니다.'}
<class 'dict'>


#### c) 스키마 기반 파싱 (PydanticOutputParser)
Pydantic 모델로 원하는 출력 구조를 정의하고, LLM이 그 구조에 맞게 출력하도록 유도함. `get_format_instructions()`로 LLM에게 가이드라인 전달. 복잡한 데이터 받을 때 안정적임.

In [16]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field # LangChain은 pydantic_v1을 내부적으로 사용

# Pydantic 모델 정의: 원하는 출력 스키마를 클래스로 명시
class Person(BaseModel):
    """사람에 대한 정보."""
    name: str = Field(..., description="그 사람의 이름")
    title: str = Field(..., description="그 사람의 직함 또는 직책.")

# PydanticOutputParser 생성 (정의한 모델을 인자로 전달)
person_parser = PydanticOutputParser(pydantic_object=Person)

print("========================================")
print("PydanticOutputParser 프롬프트 가이드라인:")
print("----------------------------------------")
# LLM에게 어떤 형식으로 출력해야 하는지 알려주는 가이드라인 생성
format_instructions = person_parser.get_format_instructions()
print(format_instructions)
print("========================================")

# 새로운 프롬프트 템플릿 생성 (시스템 메시지에 format_instructions 포함)
pydantic_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자 질문에 답하세요. 출력은 `json` 태그로 감싸주세요.\n{format_instructions}", # 여기에 가이드라인 삽입
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=format_instructions) # .partial로 format_instructions 값을 미리 채워둠

print("최종 프롬프트 템플릿 (가이드라인 포함):")
print("----------------------------------------")
print(pydantic_prompt.format(query="테슬라 창업자는 누구인가요?"))
print("========================================")

PydanticOutputParser 프롬프트 가이드라인:
----------------------------------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "사람에 대한 정보.", "properties": {"name": {"title": "Name", "description": "그 사람의 이름", "type": "string"}, "title": {"title": "Title", "description": "그 사람의 직함 또는 직책.", "type": "string"}}, "required": ["name", "title"]}
```
최종 프롬프트 템플릿 (가이드라인 포함):
----------------------------------------
System: 사용자 질문에 답하세요. 출력은 `json` 태그로 감싸주세요.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"propertie

In [17]:
# PydanticOutputParser를 포함한 체인 구성
person_chain = pydantic_prompt | llm | person_parser

# 체인 실행
pydantic_response = person_chain.invoke({"query":"테슬라 창업자는 누구인가요?"})

# 체인 응답 출력 (Pydantic 모델 객체로 반환됨)
pydantic_response
print(type(pydantic_response)) # 타입이 Person 클래스 객체인지 확인

<class '__main__.Person'>


## 3. LLM 호출, 다양하게 활용하기
LLM 객체는 `invoke` 외에도 `stream`, `batch` 등 유용한 호출 방식을 제공함.

### (1) stream: 실시간 응답 스트리밍
답변을 한 번에 다 받는 게 아니라, 생성되는 대로 토큰 단위로 바로바로 받아볼 수 있음. 사용자 경험(UX)에 좋음. `flush=True`로 즉시 출력!

In [18]:
import time 

print("스트리밍 응답 시작:")
for chunk in llm.stream("테슬라 창업자는 누구인가요?"): # 체인이 아닌 llm 객체 자체의 stream 사용
    # chunk는 AIMessageChunk 객체. content 속성에 토큰이 들어있음
    print(chunk.content, end="", flush=True)  
    # time.sleep(0.05) # 너무 빠르면 눈으로 보기 힘드니 약간의 딜레이 (선택 사항)
print("\n스트리밍 응답 종료.")

스트리밍 응답 시작:
테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타르펜닝(Mark Tarpenning)에 의해 설립되었습니다. 이후 엘론 머스크가 2004년에 투자자로 참여하게 되면서 회사의 주요 인물로 부각되었고, CEO로서 테슬라의 발전에 큰 기여
스트리밍 응답 종료.


### (2) batch: 여러 질문 한 번에 처리
질문 여러 개를 리스트로 묶어서 한 방에 처리함. API 호출을 효율적으로 관리할 수 있음.

In [19]:
questions = [
    "테슬라의 창업자는 누구인가요?",
    "리비안의 창업자는 누구인가요?",
]

# 여러 질문을 리스트로 전달하여 batch 처리
batch_responses = llm.batch(questions) # 체인이 아닌 llm 객체 자체의 batch 사용

for response in batch_responses:
    # AIMessage 객체의 pretty_print() 메서드로 보기 좋게 출력
    response.pretty_print()
    print() # 줄바꿈

================================== Ai Message ==================================

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스키너(Jeff Skilling), 그리고 이안 라이트(Ian Wright) 등 여러 사람입니다. 그러나 엘론 머스크는 테슬라의 CEO로서 가장 잘 알려져 있으며, 회사의 성장과 발전에 중요한 역할을 했습니다. 테슬라는

================================== Ai Message ==================================

리비안(Rivian)의 창업자는 RJ 스케링(RJ Scaringe)입니다. 그는 2009년에 리비안을 설립하였으며, 전기차 제조업체로서 전기 픽업트럭과 SUV를 개발하고 있습니다. 리비안은 특히 전기차 시장에서의 혁신적인 접근 방식으로 주목받고 있습니다.



## 4. Runnable: 더 유연한 체인 구성
LCEL의 핵심 `Runnable` 프로토콜을 따르는 다양한 클래스들. 복잡한 데이터 흐름이나 커스텀 로직을 체인에 통합할 때 유용함.

### (1) RunnableParallel: 병렬 실행과 데이터 매핑
여러 Runnable을 동시에 실행하거나, 입력 데이터를 딕셔너리 형태로 가공하여 다음 Runnable에 전달할 때 씀. `itemgetter`와 함께 자주 사용됨.

In [20]:
# RAG를 위해 벡터 저장소에서 문서를 검색하는 Retriever 준비
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 1}, # 가장 유사한 문서 1개 검색
)

query = "테슬라 창업자는 누구인가요?"
retrieved_docs = retriever.invoke(query)

# 검색된 문서(Document 객체 리스트)들의 page_content를 합쳐서 하나의 문자열로 만듦
retrieved_docs_text = "\n".join([doc.page_content for doc in retrieved_docs])

pprint(retrieved_docs_text)

('텍사스주 오스틴에 본사를 둔 테슬라(Tesla, Inc.)는 미국의 대표적인 전기 자동차 제조업체입니다. 2003년 마틴 '
 '에버하드(Martin Eberhard, CEO)와 마크 타페닝(Marc Tarpenning, CFO)이 설립한 테슬라는 2004년 일론 '
 '머스크(Elon Musk)의 적극적인 참여를 받았습니다. 페이팔(PayPal)과 짚투(Zip2)의 공동 창립자인 머스크는 최대 주주이자 '
 '회장이 되어 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 저명한 물리학자이자 전기 공학자인 니콜라 테슬라(Nikola Tesla)의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.')


In [21]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter # 딕셔너리에서 특정 키의 값을 가져올 때 사용

# RunnableParallel 구성: 입력 딕셔셔너리에서 'context'와 'question' 키의 값을 그대로 가져와 새로운 딕셔너리 생성
setup = RunnableParallel(
    context=itemgetter("context") , 
    question=itemgetter("question")
)

# 실행: 입력으로 딕셔너리를 전달
runnable_parallel_output = setup.invoke({"context": retrieved_docs_text, "question": query})
runnable_parallel_output

{'context': '텍사스주 오스틴에 본사를 둔 테슬라(Tesla, Inc.)는 미국의 대표적인 전기 자동차 제조업체입니다. 2003년 마틴 에버하드(Martin Eberhard, CEO)와 마크 타페닝(Marc Tarpenning, CFO)이 설립한 테슬라는 2004년 일론 머스크(Elon Musk)의 적극적인 참여를 받았습니다. 페이팔(PayPal)과 짚투(Zip2)의 공동 창립자인 머스크는 최대 주주이자 회장이 되어 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 저명한 물리학자이자 전기 공학자인 니콜라 테슬라(Nikola Tesla)의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.',
 'question': '테슬라 창업자는 누구인가요?'}

### (2) RunnablePassthrough: 입력 그대로 전달
입력값을 다음 단계로 그대로 넘기거나, `RunnableParallel`과 함께 사용하여 특정 키에 원본 입력을 할당할 때 유용함.

In [22]:
from langchain_core.runnables import RunnablePassthrough

# RunnableParallel 내에서 RunnablePassthrough 사용 예시
pass_through_setup = RunnableParallel(
    original_input=RunnablePassthrough(), # 입력을 그대로 'original_input' 키에 할당
) # 여기에 다른 Runnable들을 추가하여 병렬 처리 가능

pass_through_setup.invoke({"query":"테슬라 창업자는 누구인가요?"})

{'original_input': {'query': '테슬라 창업자는 누구인가요?'}}

### (3) RunnableLambda: 파이썬 함수도 체인에 착!
간단한 파이썬 함수를 LCEL 체인 안에 컴포넌트처럼 넣을 수 있음. 커스텀 로직 추가에 매우 유용함.

In [23]:
from langchain_core.runnables import RunnableLambda

# 간단한 단어 수 세는 함수 정의
def count_num_words(text_input):
    if isinstance(text_input, dict) and 'query' in text_input: # 입력이 딕셔너리인 경우 'query' 키 사용
        return len(text_input['query'].split())
    elif isinstance(text_input, str): # 입력이 문자열인 경우
        return len(text_input.split())
    return 0

# RunnableParallel과 RunnableLambda 조합
lambda_setup = RunnableParallel(
    question=RunnablePassthrough(), # 입력을 'question' 키에 그대로 전달
    word_count=RunnableLambda(count_num_words), # count_num_words 함수를 Runnable로 만들어 'word_count' 키에 할당
)

lambda_setup.invoke("테슬라 창업자는 누구인가요?") # 문자열 입력도 가능

{'question': '테슬라 창업자는 누구인가요?', 'word_count': 3}

## 5. 실전! RAG 파이프라인 구축
지금까지 배운 LCEL 컴포넌트들을 조합하여 질문에 대해 관련 문서를 찾아 답변하는 RAG 파이프라인을 만듦.

### (1) RAG용 프롬프트 템플릿
RAG의 핵심 프롬프트. LLM에게 '주어진 Context 안에서만 답변하고, 모르면 모른다고 해!'라고 지시하는 게 중요함. 외부 지식 사용 방지.

In [24]:
from langchain.prompts import ChatPromptTemplate # 이미 위에서 임포트 했지만, 명시적으로 다시 보여줌

template = """Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]
"""

rag_prompt = ChatPromptTemplate.from_template(template)

rag_prompt.pretty_print() # 생성된 RAG 프롬프트 구조 확인

================================ Human Message =================================

Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]



### (2) 리트리버 체인: 문서 가져오고 포맷팅
질문과 가장 유사한 문서를 벡터 저장소에서 찾아옴 (`retriever`). 찾은 문서들(Document 객체 리스트)을 LLM이 이해하기 쉬운 하나의 문자열로 합침 (`format_docs`).

In [25]:
# 벡터 저장소 기반 리트리버 (k=2: 가장 유사한 문서 2개 검색)
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

# 검색된 Document 객체 리스트를 하나의 문자열로 포맷팅하는 함수
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# 리트리버 체인: retriever | format_docs (RunnableLambda로 함수를 감싸도 동일)
retriever_chain = retriever | RunnableLambda(format_docs)
# 또는 retriever_chain = RunnableLambda(lambda x: format_docs(retriever.invoke(x))) 이런식으로 한번에 구성도 가능

# 리트리버 체인 테스트
test_retrieved_text = retriever_chain.invoke("테슬라 창업자는 누구인가요?")
pprint(test_retrieved_text)

('텍사스주 오스틴에 본사를 둔 테슬라(Tesla, Inc.)는 미국의 대표적인 전기 자동차 제조업체입니다. 2003년 마틴 '
 '에버하드(Martin Eberhard, CEO)와 마크 타페닝(Marc Tarpenning, CFO)이 설립한 테슬라는 2004년 일론 '
 '머스크(Elon Musk)의 적극적인 참여를 받았습니다. 페이팔(PayPal)과 짚투(Zip2)의 공동 창립자인 머스크는 최대 주주이자 '
 '회장이 되어 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 저명한 물리학자이자 전기 공학자인 니콜라 테슬라(Nikola Tesla)의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년 대비 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 '
 '테슬라의 누적 글로벌 판매량은 4,962,975대를 넘어섰습니다. SMT 패키징(SMT Packaging)에 따르면, 테슬라의 2023년 '
 '판매량은 글로벌 전기 자동차 시장의 약 12.9%를 차지했습니다.')


### (3) RAG 체인 완성: 모든 조각 맞추기
이제 모든 걸 연결함:
1.  사용자 질문(`question`)을 받음.
2.  `retriever_chain`을 사용해 질문과 관련된 문서(`context`)를 가져옴.
3.  `RunnablePassthrough`를 사용해 원본 질문을 그대로 전달함.
4.  이 `context`와 `question`을 `rag_prompt`에 넣어 완성된 프롬프트를 만듦.
5.  이 프롬프트를 `llm`에 전달하여 답변 생성.
6.  `StrOutputParser`로 LLM의 답변에서 텍스트만 추출함.

In [26]:
from langchain_core.output_parsers import StrOutputParser # 이미 임포트됨
from langchain_openai import ChatOpenAI # 이미 임포트됨

# RAG용 LLM 모델 (temperature=0으로 좀 더 사실 기반 답변 유도)
rag_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=150)

# 전체 RAG 체인 구성
rag_chain = (
    RunnableParallel(
        context=retriever_chain,  # retriever_chain의 출력이 'context' 키로 들어감
        question=RunnablePassthrough() # 원본 질문이 'question' 키로 들어감
    )
    | rag_prompt # context와 question을 받아 프롬프트 완성
    | rag_llm    # 완성된 프롬프트를 LLM에 전달
    | StrOutputParser() # LLM 응답(AIMessage)에서 문자열만 추출
)

# RAG 체인 실행
query = "테슬라 창업자는 누구인가요?"
final_response = rag_chain.invoke(query)


In [27]:
# 최종 결과 출력
final_response

'마틴 에버하드와 마크 타페닝입니다.'